In [ ]:
!pip install scikit-fuzzy > /dev/null 2>&1

In [ ]:
import numpy as np
import skfuzzy
from skfuzzy import control


# Universe of Discourse
IP_universe_of_discourse = np.arange(0, 100, 1)
OP_universe_of_discourse = np.arange(0, 60, 1)

# Inputs to FLC
dirt = control.Antecedent(universe=IP_universe_of_discourse, label="dirt")
dirt["SD"] = skfuzzy.membership.trimf(dirt.universe, [0, 0, 50])
dirt["MD"] = skfuzzy.membership.trimf(dirt.universe, [0, 50, 100])
dirt["LD"] = skfuzzy.membership.trimf(dirt.universe, [50, 100, 100])
# dirt.view()

grease = control.Antecedent(universe=IP_universe_of_discourse, label="grease")
grease["SG"] = skfuzzy.membership.trimf(grease.universe, [0, 0, 50])
grease["MG"] = skfuzzy.membership.trimf(grease.universe, [0, 50, 100])
grease["LG"] = skfuzzy.membership.trimf(grease.universe, [50, 100, 100])
# grease.view()

# Output from FLC
wash_time = control.Consequent(universe=OP_universe_of_discourse, label="wash_time")
wash_time.defuzzify_method = "mom"
wash_time["VS"] = skfuzzy.membership.trimf(wash_time.universe, [0, 0, 10])
wash_time["S"] = skfuzzy.membership.trimf(wash_time.universe, [0, 10, 25])
wash_time["M"] = skfuzzy.membership.trimf(wash_time.universe, [10, 25, 40])
wash_time["L"] = skfuzzy.membership.trimf(wash_time.universe, [25, 40, 60])
wash_time["VL"] = skfuzzy.membership.trimf(wash_time.universe, [40, 60, 60])


# Defining the rules
rule_1 = control.Rule(
    antecedent=(dirt["SD"] & grease["SG"]), consequent=wash_time["VS"]
)
rule_2 = control.Rule(dirt["SD"] & grease["MG"], wash_time["M"])
rule_3 = control.Rule(dirt["SD"] & grease["LG"], wash_time["L"])
rule_4 = control.Rule(dirt["MD"] & grease["SG"], wash_time["S"])
rule_5 = control.Rule(dirt["MD"] & grease["MG"], wash_time["M"])
rule_6 = control.Rule(dirt["MD"] & grease["LG"], wash_time["L"])
rule_7 = control.Rule(dirt["LD"] & grease["SG"], wash_time["M"])
rule_8 = control.Rule(dirt["LD"] & grease["MG"], wash_time["L"])
rule_9 = control.Rule(dirt["LD"] & grease["LG"], wash_time["VL"])

wash_control_sys = control.ControlSystem(
    rules=[rule_1, rule_2, rule_3, rule_4, rule_5, rule_6, rule_7, rule_8, rule_9]
)

washing_machine = control.ControlSystemSimulation(control_system=wash_control_sys)

dirt_list = [80]
grease_list = [90]

for dirt_val, grease_val in zip(dirt_list, grease_list):
    washing_machine.inputs({"dirt": dirt_val, "grease": grease_val})

    washing_machine.compute()
    washing_machine.print_state()
    print(
        "\n\tWash-time for {} % Dirt & {} % Grease: {}".format(
            dirt_val, grease_val, washing_machine.output["wash_time"]
        )
    )

wash_time.view(sim=washing_machine)

 Antecedents 


TypeError: ignored